In [1]:
pip install requests pandas pymongo tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
import json
import pandas as pd
import time
from pymongo import MongoClient
from tqdm import tqdm
import re

In [3]:
# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")  
db = client["steam_reviews"] 
collection = db["analyzed_reviews"]  


In [4]:
# OpenRouter API Key
API_KEY = "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"
API_URL = "https://openrouter.ai/api/v1/chat/completions"

In [ ]:
def analyze_review(review_text):

    sprompt = """
    You are an AI assistant analyzing video game reviews using Aspect-Based Sentiment Analysis (ABSA).
    Your task: Analyze the following review and identify the mentioned aspects.
    For each aspect, classify the sentiment as positive, negative, neutral, or not mentioned, and provide a reason.

    Aspects to Analyze:

    Gameplay : including but not limited to : Covers mechanics, skill trees, controls, and overall game feel.
    Graphics Fidelity : including but not limited to : Evaluates visual quality, including realism, art style, and graphical polish. Any graphical bugs should be categorized under Technical.
    Story : including but not limited to : Assesses quests, dialogue quality, character development, and overall narrative structure.
    Technical : including but not limited to : Focuses on performance issues, bugs, crashes, optimization, and stability.
    World : including but not limited to : Examines world-building, city/environment design, immersion, and level design.
    Multiplayer : including but not limited to : Reviews social features, matchmaking, co-op functionality, and overall multiplayer experience.

💡 Instructions: Follow this exact order when analyzing aspects. Even if an aspect is not mentioned in the review, still include it and provide a clear reason why it was not mentioned.

     The output format should be:
    {
      "aspects": [
        {
          "aspect": "<aspect_name>",
          "sentiment": "<positive/negative/neutral/not mentioned>",
          "reason": "<brief reason>"
        },
        ...
      ]
    }

    Ensure the output is in **strict JSON format**.
    """
    payload = {
        "model": "google/gemini-2.0-flash-exp:free",  
        "messages": [
            {"role": "system", "content": sprompt},
            {"role": "user", "content": review_text}
        ],
        "temperature": 0.2,
        "stream": False 
    }
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    try:
        response = requests.post(API_URL, headers=headers, json=payload)
        response.raise_for_status()  
        llm_content = response.json()["choices"][0]["message"]["content"]
        print("🔹 Raw LLM Output:", llm_content)  
        cleaned_json = re.sub(r"```json\n(.*?)\n```", r"\1", llm_content, flags=re.DOTALL)
        analysis_json = json.loads(cleaned_json)
        return analysis_json

    except requests.exceptions.RequestException as e:
        print("Request Error:", str(e))
        return {"error": str(e)}

    except json.JSONDecodeError:
        print("JSON Decode Error. Response was not valid JSON.")
        return {"error": "Invalid JSON response"}


In [6]:
def process_reviews(file_path):
    df = pd.read_csv(file_path)
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        review_text = row["review"]
        analysis_result = analyze_review(review_text)
        final_data = {
            "app_id": row["app_id"],
            "recommended": row["recommended"],
            "date_posted": row["date_posted"],
            "review_text": review_text,
            "analysis": analysis_result
        }
        collection.insert_one(final_data)
        time.sleep(3)
    print("Stored in mongodb")


In [7]:
process_reviews("steamreviews.csv")

  0%|          | 0/3 [00:00<?, ?it/s]

🔹 Raw LLM Output: ```json
{
  "aspects": [
    {
      "aspect": "Gameplay",
      "sentiment": "negative",
      "reason": "The review criticizes the lack of depth in gameplay mechanics, comparing it unfavorably to a 'first person GTA knockoff' and a 'looter-shooter with GTA elements'. It mentions skill trees and combat options but notes a lack of significant depth."
    },
    {
      "aspect": "Graphics Fidelity",
      "sentiment": "not mentioned",
      "reason": "The review does not mention the graphical quality or art style of the game."
    },
    {
      "aspect": "Story",
      "sentiment": "negative",
      "reason": "The review mentions that choices and 'life paths' are mostly inconsequential, suggesting a lack of meaningful impact from the narrative elements."
    },
    {
      "aspect": "Technical",
      "sentiment": "not mentioned",
      "reason": "The review does not mention any performance issues, bugs, or optimization problems."
    },
    {
      "aspect": "World"

 33%|███▎      | 1/3 [00:08<00:16,  8.07s/it]

🔹 Raw LLM Output: ```json
{
  "aspects": [
    {
      "aspect": "Gameplay",
      "sentiment": "positive",
      "reason": "The reviewer mentions the gameplay is a lot of fun, the guns pack a huge punch, and melee with katanas is fun. They also praise the decision-making and quest design."
    },
    {
      "aspect": "Graphics Fidelity",
      "sentiment": "neutral",
      "reason": "The reviewer states that graphics are important but less so than player agency, good writing, and fun gameplay."
    },
    {
      "aspect": "Story",
      "sentiment": "positive",
      "reason": "The reviewer states that narratively the game excels, the dialogue is well written and acted, and the characters are hard to not fall in love with."
    },
    {
      "aspect": "Technical",
      "sentiment": "negative",
      "reason": "The reviewer mentions crashing issues initially and that the game is plagued with bugs, even after a hotfix. They describe them as 'immersion raping bugs'."
    },
    {
   

 67%|██████▋   | 2/3 [00:16<00:08,  8.02s/it]

🔹 Raw LLM Output: ```json
{
  "aspects": [
    {
      "aspect": "Gameplay",
      "sentiment": "neutral",
      "reason": "The review mentions 'next-gen gameplay? Nope' but doesn't elaborate on specific gameplay mechanics, controls, or skill trees. Therefore, the sentiment is neutral."
    },
    {
      "aspect": "Graphics Fidelity",
      "sentiment": "negative",
      "reason": "The review states 'Next-gen graphics? Hmm.. no, I don't think so,' indicating a negative sentiment towards the game's graphics fidelity."
    },
    {
      "aspect": "Story",
      "sentiment": "positive",
      "reason": "The review expresses a positive sentiment towards the story, describing it as 'great story' and 'rock solid' and highlighting the multiple paths and replayability offered by the Phantom Liberty expansion."
    },
    {
      "aspect": "Technical",
      "sentiment": "negative",
      "reason": "The review explicitly mentions the game being 'Unpolished and buggy? Yep' and 'Glitchy and res

100%|██████████| 3/3 [00:24<00:00,  8.24s/it]

Stored in mongodb
